In [10]:
import sys
# enable importing the modules from probcalkge
sys.path.append('../')
sys.path.append('../probcalkge')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from probcalkge import Experiment, ExperimentResult
from probcalkge import get_calibrators
from probcalkge import get_datasets,  get_kgemodels
from probcalkge import brier_score, negative_log_loss, ks_error, ece

from ampligraph.evaluation import generate_corruptions_for_fit, generate_corruptions_for_eval

In [2]:
cals = get_calibrators()
kges = get_kgemodels()
ds = get_datasets()

WARNING - All triples will be processed in the same batch (batches_count=1). When processing large graphs it is recommended to batch the input knowledge graph instead.


In [3]:
exp = Experiment(
    cals=[cals.uncal, cals.platt, cals.isot, cals.beta, cals.histbin, cals.temperature], 
    datasets=[ds.fb13, ds.wn18, ds.yago39, ds.dp50, ds.nations, ds.kinship, ds.umls], 
    kges=[kges.transE, kges.complEx, kges.distMult, kges.hoLE], 
    metrics=[ece]
    )

In [5]:
exp.load_trained_kges('../saved_models/')
exp.run_with_trained_kges()

Loaded models:
{'DBpedia50': {'ComplEx': <ampligraph.latent_features.models.ComplEx.ComplEx object at 0x000001622DDE1EC8>,
               'DistMult': <ampligraph.latent_features.models.DistMult.DistMult object at 0x000001621A964408>,
               'HolE': <ampligraph.latent_features.models.HolE.HolE object at 0x0000016227640F88>,
               'TransE': <ampligraph.latent_features.models.TransE.TransE object at 0x000001622DE06048>},
 'DBpedia_ET': OrderedDict([('ComplEx',
                             <ampligraph.latent_features.models.ComplEx.ComplEx object at 0x000001622427F048>),
                            ('DistMult',
                             <ampligraph.latent_features.models.DistMult.DistMult object at 0x0000016228C13048>),
                            ('TransE',
                             <ampligraph.latent_features.models.TransE.TransE object at 0x0000016229DCA048>)]),
 'FB13k': {'ComplEx': <ampligraph.latent_features.models.ComplEx.ComplEx object at 0x000001622DE06188>,

In [25]:
from probcalkge import EmbeddingModel, Calibrator

class ProbKGE:
    def __init__(self, kge: EmbeddingModel, cal: Calibrator) -> None:
        self.kge = kge
        self.cal = cal

    def fit(self, X_train, y_train, X_valid, y_valid):
        self.kge.fit(X_train, y_train)
        scores_valid = self.kge.predict(X_valid)
        self.cal.fit(scores_valid, y_valid)

    def predict(self, X):
        scores = self.kge.predict(X)
        probs = self.cal.predict(scores)
        return probs


In [28]:
# def eval_LP() :

from ampligraph.evaluation import create_mappings, rank_score, mrr_score, hits_at_n_score

data = ds.fb13
filter_triples = np.concatenate((data.X_train, data.X_valid, data.X_test))
model = ProbKGE(exp.trained_kges[data.name]['ComplEx'], 
            exp.trained_cals[data.name]['ComplEx']['BetaCalibrator'])

def eval_LP(model: ProbKGE, X_test: np.ndarray, eta: int = 5):
    r2idx, e2idx = create_mappings(X_test)
    idx2e = { v:k for k,v in e2idx.items()}
    idx2r = { v:k for k,v in r2idx.items()}
    X_test_id = np.array([(e2idx[trp[0]], r2idx[trp[1]], e2idx[trp[2]]) for trp in X_test])
    X_neg_id = generate_corruptions_for_fit(X_test_id, eta=eta).eval(session=tf.Session())
    X_neg_id = X_neg_id.reshape([X_test_id.shape[0], eta, X_test_id.shape[1]])
    print(X_test.shape, X_neg_id.shape)
    X_negs = []
    for row in X_neg_id:
        tmp = []
        for t in row:
            # print(t)
            tmp.append((idx2e[t[0]], idx2r[t[1]], idx2e[t[2]]))
        X_negs.append(tmp)
    X_negs = np.array(X_negs)
    ranks = []
    for pos, negs in zip(X_test, X_negs):
        y_true = np.array([1] + [0]*eta)
        y_pred = np.array([model.predict(pos)] + [model.predict(neg) for neg in negs])
        ranks.append(rank_score(y_true, y_pred))
    ranks = np.array(ranks)
    return mrr_score(ranks)
    # print(X_test.shape, X_neg_id.shape)
eval_LP(model, data.X_test)

(47464, 3) (47464, 5, 3)


KeyboardInterrupt: 

In [18]:
ds.kinship.X_test[0]

array([59,  5, 64])